# ⚙️ Interactive Controls in Jupyter (ipywidgets)

Interactive widgets let you **play** with parameters (sliders, dropdowns, checkboxes) and **see results update instantly** — perfect for exploring energy data (demand vs. temperature, weekly profiles, generation mixes).

---

# Setup & Imports 🛠️

```python
# If needed:
# !pip install ipywidgets

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, interactive_output
import ipywidgets as widgets

# For nicer plots
plt.rcParams["figure.figsize"] = (9, 4)
plt.rcParams["axes.grid"] = True

# 🔹 Quick demo (no boilerplate)

In [2]:
@interact(a=(0, 10, 1), b=(0, 10, 1))
def add(a=3, b=4):
    print(f"{a} + {b} = {a+b}")

interactive(children=(IntSlider(value=3, description='a', max=10), IntSlider(value=4, description='b', max=10)…

In [3]:
@interact(source=["Solar", "Wind", "Hydro"], normalized=False)
def show_config(source="Solar", normalized=False):
    print(f"Source: {source}, normalized={normalized}")



interactive(children=(Dropdown(description='source', options=('Solar', 'Wind', 'Hydro'), value='Solar'), Check…

# Energy Example: Demand vs Temperature (U-shape) 🌡️⚡

Interactively explore how parameters change the curve.


In [4]:
x = np.linspace(-15, 32, 300)  # temperature °C

@interact(center=(0, 25, 1), scale=(1, 15, 1), noise=(0.0, 100.0, 5.0))
def demand_vs_temp(center=15, scale=6, noise=40.0):
    rng = np.random.default_rng(42)
    y = 1200 + scale*(x - center)**2 + rng.normal(0, noise, size=len(x))
    plt.plot(x, y, label="Demand model")
    plt.title("Demand vs Temperature (interactive model)")
    plt.xlabel("Temp (°C)"); plt.ylabel("Demand [MW]")
    plt.legend(); plt.show()

interactive(children=(IntSlider(value=15, description='center', max=25), IntSlider(value=6, description='scale…

> 🧠 **Interpretation:** moving the **center** mimics comfort temperature; **scale** controls sensitivity; **noise** simulates unexplained variability.


---

# Interactive Time-Window Filter with DateRangeSlider ⏳

Filter a synthetic hourly demand series by an adjustable date range.

In [6]:
# Synthetic hourly demand for 60 days
idx = pd.date_range("2024-01-01", periods=24*60, freq="h")
demand = 1200 + 250*np.sin(2*np.pi*(idx.hour/24)) + np.random.normal(0, 45, len(idx))
df = pd.DataFrame({"time": idx, "demand": demand}).set_index("time")

start = widgets.SelectionRangeSlider(
    options=[pd.Timestamp(t) for t in df.index],
    index=(0, len(df.index)-1),
    description="Window",
    layout=widgets.Layout(width="95%"),
    continuous_update=False
)

def plot_window(window):
    s, e = window
    sub = df.loc[s:e]
    sub.plot(y="demand", legend=False)
    plt.title(f"Demand window: {s.date()} → {e.date()}")
    plt.xlabel("Time"); plt.ylabel("Demand [MW]")
    plt.show()

widgets.interact(plot_window, window=start);

interactive(children=(SelectionRangeSlider(continuous_update=False, description='Window', index=(0, 1439), lay…

> ⛽ **Tip:** `continuous_update=False` prevents constant redraw while dragging — smoother for big data.

---

# Generation Mix Playground (Checkboxes) 🌞💨💧

Toggle sources on/off and scale them live.

In [7]:
days = pd.date_range("2024-03-01", periods=21, freq="D")
solar = np.clip(200 + 60*np.sin(2*np.pi*(days.dayofyear/365)), 120, 320)
wind  = 300 + 120*np.sin(2*np.pi*(days.dayofyear/10) + 1)
hydro = 500 + np.random.normal(0, 30, len(days))

mix = pd.DataFrame({"day": days, "Solar": solar, "Wind": wind, "Hydro": hydro}).set_index("day")

use_solar = widgets.Checkbox(value=True, description="Solar")
use_wind  = widgets.Checkbox(value=True, description="Wind")
use_hydro = widgets.Checkbox(value=True, description="Hydro")
scale     = widgets.FloatSlider(value=1.0, min=0.5, max=1.5, step=0.05, description="Scale", readout_format=".2f")

def update(use_solar, use_wind, use_hydro, scale):
    comp = []
    if use_solar: comp.append("Solar")
    if use_wind:  comp.append("Wind")
    if use_hydro: comp.append("Hydro")
    if not comp:
        plt.figure(); plt.title("No sources selected"); plt.show(); return
    (mix[comp]*scale).plot.area()
    plt.title("Generation Mix (interactive)")
    plt.xlabel("Day"); plt.ylabel("Energy [MWh]")
    plt.show()

ui = widgets.HBox([use_solar, use_wind, use_hydro, scale])
out = widgets.interactive_output(update, {"use_solar":use_solar, "use_wind":use_wind, "use_hydro":use_hydro, "scale":scale})
display(ui, out)

Output()

> 🧩 **Pattern:** `interactive_output(function, controls_dict)` gives flexible layout control (widgets in one box, plot below).

---

# Weekday vs Weekend Load Profiles (ToggleButtons) 📅

Switch profile type and smoothing window.

In [8]:
hours = np.arange(24)
weekday = 900 + 220*np.exp(-0.5*((hours-8)/2.2)**2) + 260*np.exp(-0.5*((hours-18)/2.7)**2)
weekend = 820 + 140*np.exp(-0.5*((hours-11)/3.1)**2) + 120*np.exp(-0.5*((hours-19)/3.6)**2)
rng = np.random.default_rng(0)
weekday += rng.normal(0, 15, size=24); weekend += rng.normal(0, 15, size=24)

profile = pd.DataFrame({"hour": hours, "Weekday": weekday, "Weekend": weekend}).set_index("hour")

mode = widgets.ToggleButtons(options=["Weekday","Weekend"], description="Day type")
smooth = widgets.IntSlider(value=1, min=1, max=5, step=1, description="Smooth k")

def plot_profile(mode, smooth):
    y = profile[mode].copy()
    if smooth > 1:
        y = pd.Series(y).rolling(smooth, min_periods=1, center=True).mean().values
    plt.plot(hours, profile[mode], "o-", alpha=0.4, label="raw")
    plt.plot(hours, y, "o-", label=f"smoothed (k={smooth})")
    plt.title(f"{mode} Load Profile")
    plt.xlabel("Hour"); plt.ylabel("Demand [MW]"); plt.legend(); plt.show()

display(widgets.HBox([mode, smooth]))
out = widgets.interactive_output(plot_profile, {"mode":mode, "smooth":smooth})
display(out)

Output()

---

# Parameter Fitting Sandbox (Manual) 🧪

Manually tune a simple linear model to see effects on residuals.

In [10]:
# Synthetic: demand vs temp (roughly linear in a limited range)
rng = np.random.default_rng(3)
n = 300
temp = rng.uniform(-5, 25, n)
demand = 1400 - 20*temp + rng.normal(0, 40, n)
df_lin = pd.DataFrame({"temp":temp, "demand":demand})

alpha = widgets.FloatSlider(value=1400, min=1000, max=1800, step=10, description="Intercept")
beta  = widgets.FloatSlider(value=-20,  min=-60,  max=20,   step=2,  description="Slope")

def fit_demo(alpha, beta):
    yhat = alpha + beta*df_lin["temp"]
    resid = df_lin["demand"] - yhat
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    ax[0].scatter(df_lin["temp"], df_lin["demand"], alpha=0.5, label="data")
    ax[0].plot(np.sort(df_lin["temp"]), alpha + beta*np.sort(df_lin["temp"]), color="red", label="model")
    ax[0].set_title("Demand vs Temp (manual model)"); ax[0].set_xlabel("Temp (°C)"); ax[0].set_ylabel("Demand [MW]"); ax[0].legend()
    ax[1].hist(resid, bins=20)
    ax[1].set_title(f"Residuals (MAE={resid.abs().mean():.1f})"); ax[1].set_xlabel("Error [MW]")
    plt.show()

display(alpha, beta)
out = widgets.interactive_output(fit_demo, {"alpha":alpha, "beta":beta})
display(out)

FloatSlider(value=1400.0, description='Intercept', max=1800.0, min=1000.0, step=10.0)

FloatSlider(value=-20.0, description='Slope', max=20.0, min=-60.0, step=2.0)

Output()


---

# Pro Tips for Smooth Interactivity 🚀

* **Turn off live updates** while dragging heavy sliders: `continuous_update=False`.
* **Debounce text inputs** with `Text(value="", continuous_update=False)`.
* **Cache data** outside the callback; only recompute what’s necessary inside.
* **Prefer vectorized ops** (NumPy/Pandas) over Python loops in callbacks.
* For large plots, consider **downsampling** or using Plotly for fast interactivity.

---

# Quick Widget Gallery 🎛️


In [11]:
widgets.VBox([
    widgets.IntSlider(description="Int"),
    widgets.FloatSlider(description="Float", step=0.1),
    widgets.Dropdown(options=["Solar","Wind","Hydro"], description="Source"),
    widgets.RadioButtons(options=["MAE","RMSE","MAPE"], description="Metric"),
    widgets.Checkbox(description="Normalize", value=False),
    widgets.Text(description="Label"),
    widgets.ColorPicker(concise=True, description="Color"),
])

> 📚 Docs: `help(widgets)` or the ipywidgets examples in Jupyter.

----
# Minimal Pattern to Reuse 🧩

Structure your interactive cell like this:

```python
# 1) Build widgets
w1 = widgets.FloatSlider(..., continuous_update=False)
w2 = widgets.Dropdown(...)

# 2) Define update function
def update(param1, param2):
    # compute / filter / plot
    pass

# 3) Wire up & display
ui = widgets.HBox([w1, w2])
out = widgets.interactive_output(update, {"param1": w1, "param2": w2})
display(ui, out)
```

